In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from peft import get_peft_model, PrefixTuningConfig, TaskType

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
print(tokenizer.pad_token)
print(tokenizer.eos_token)

tokenizer.pad_token = tokenizer.eos_token

None
<|endoftext|>


In [4]:
## 모델 파라미터 고정.
for param in model.parameters():
    param.requires_grad = False

In [5]:
# Prefix Tuning 설정 생성
prefix_config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM, ## task
    num_virtual_tokens=20 ## 가상토큰(prefix 토큰) 수
)

# PEFT 모델 생성
model = get_peft_model(model, prefix_config)

In [11]:
input_texts = [
    "오늘은 날씨가 정말 좋네요.",
    "인공지능은 미래를 바꿀 기술입니다.",
    "프리픽스 튜닝을 공부하고 있습니다."
]

# 토큰화 및 텐서 변환
inputs = tokenizer(
    input_texts,
    return_tensors='pt',
    padding=True,          # 시퀀스 패딩
    truncation=True        # 시퀀스 길이 제한
)
labels = inputs['input_ids']

# 옵티마이저 설정 (프리픽스 파라미터만 업데이트)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [13]:
model.train()
epochs = 3

for epoch in range(epochs):
    optimizer.zero_grad()
    
    # 모델 출력 얻기
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    
    # 역전파 및 파라미터 업데이트
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


Epoch 1/3, Loss: 13.989359855651855
Epoch 2/3, Loss: 13.648009300231934
Epoch 3/3, Loss: 13.478689193725586


In [14]:
model.eval()

prompt_text = "인공지능이란"
input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

# 모델 출력 생성
with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        max_length=50,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # 패딩 토큰 ID 설정
    )

# 결과 디코딩 및 출력
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/peft/peft_model.py:1685: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


인공지능이란..

j.-a-and.‑J.JJKJLKLJHJSheLLHLlJShLSh
